In [ ]:
import ray
import os
# import pandas as pd
# import pyarrow.csv as pv
# import pyarrow as pa

# Assuming sample_edges_filepath is defined from a previous cell
# sample_edges_filepath = os.path.join(output_dir, "sample_edges.txt")

# Initialize Ray if not already initialized
# if not ray.is_initialized():
ray.init(address="ray://35.202.248.118:10001")


2025-07-24 22:25:19,433	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-24 22:25:19,560	INFO client_builder.py:242 -- Passing the following kwargs to ray.init() on the server: log_to_driver


Python version:,3.10.12
Ray version:,2.48.0
Dashboard:,http://10.128.0.8:8265


Put failed:
Put failed:
Put failed:
2025-07-24 22:39:02,904	WARNING dataclient.py:403 -- Encountered connection issues in the data channel. Attempting to reconnect.


In [6]:
import os
# import pandas as pd
import pyarrow.csv as pv
import pyarrow as pa
import ray

# Get the absolute path of the edges file
sample_edges_filepath = os.path.abspath("edges.tsv")

print(
    f"Loading sample edges from {os.path.basename(sample_edges_filepath)} into a Ray Dataset..."
)

try:
    # Ensure Ray is initialized and connected
    # if not ray.is_initialized():
    #     try:
    #         ray.init(address="ray://35.202.248.118:10001")
    #     except Exception as e:
    #         print(f"Failed to connect to Ray cluster: {e}")
    #         raise

    # Use pyarrow.csv.read_csv with read_options to specify the delimiter
    read_options = pv.ReadOptions(column_names=["source", "target"])
    parse_options = pv.ParseOptions(delimiter="\t")
    convert_options = pv.ConvertOptions(
        column_types={"source": pa.string(), "target": pa.string()}
    )

    ds = ray.data.read_csv(
        sample_edges_filepath,
        read_options=read_options,
        parse_options=parse_options,
        convert_options=convert_options,
    )

    print("Ray Dataset created. Here are the first few rows:")
    ds.show(5)

    print("\nCreating incidence list using Ray Data's groupby and map_groups...")

    # Group by 'source'
    grouped_ds = ds.groupby("source")

    # Define output schema for better performance
    output_schema = {
        "source": pa.string(),
        "targets": pa.list_(pa.string())
    }

    # Apply a function to each group to collect targets into a list
    incidence_list_ds = grouped_ds.map_groups(
        lambda group_df: 
            {
                "source": [group_df["source"].iloc[0]],
                "targets": [group_df["target"].tolist()],
            },
        schema=output_schema  # Explicitly define the output schema
    )

    print("Incidence list created using Ray Data. Showing first 5 entries:")
    incidence_list_ds.show(5)

except FileNotFoundError:
    print(
        f"Error: The file {sample_edges_filepath} was not found. Please ensure it exists at: {os.path.dirname(sample_edges_filepath)}"
    )
except ray.exceptions.RaySystemError as e:
    print(f"Ray cluster connection error: {e}")
    print("Please ensure the Ray cluster is running and accessible.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Stack trace:", e.__class__.__name__)

Loading sample edges from edges.tsv into a Ray Dataset...


KeyboardInterrupt: 